In [1]:
import torch
from torch.nn import Sequential, Linear, ReLU, BatchNorm1d
from torch_geometric.nn import SAGEConv, global_mean_pool
import torch.nn.functional as F
from torch.nn import Linear, Sequential, ReLU, BatchNorm1d
from torch_geometric.data import DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
import numpy as np
import os
import pandas as pd
import glob
import random

/mnt/USR_DATA/ChenGeng/anaconda3/envs/chemprop/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Model construction for GraphSAGE

In [2]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.best_epoch = None

    def __call__(self, val_loss, model, epoch):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.best_epoch = epoch
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_epoch = epoch
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

class GraphSAGE(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GraphSAGE, self).__init__()
        self.build_model(num_node_features, num_classes)

    def build_model(self, num_node_features, num_classes):
        self.conv1 = SAGEConv(num_node_features, 64)
        self.bn1 = BatchNorm1d(64)
        self.conv2 = SAGEConv(64, 64)
        self.bn2 = BatchNorm1d(64)

        self.fc1 = Linear(64, 64)
        self.fc2 = Linear(64, 32)
        self.fc3 = Linear(32, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = F.relu(self.conv1(x, edge_index))
        x = self.bn1(x)
        x = F.dropout(x, p=0.2, training=self.training)

        x = F.relu(self.conv2(x, edge_index))
        x = self.bn2(x)
        x = global_mean_pool(x, batch)  

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

## Model training and evaluation

In [3]:
def r2_score(predictions, targets):
    target_mean = torch.mean(targets)
    ss_tot = torch.sum((targets - target_mean) ** 2)
    ss_res = torch.sum((targets - predictions) ** 2)
    return 1 - ss_res / ss_tot

def pcc_score(predictions, targets):
    predictions = predictions.squeeze().detach().numpy()
    targets = targets.squeeze().detach().numpy()
    pcc, _ = pearsonr(predictions, targets)
    return pcc

def train(model, optimizer, loader, device):
    model.train()
    total_loss = 0
    targets = []
    predictions = []
    for data in loader:
        optimizer.zero_grad()
        data.x = data.x.float()
        data = data.to(device)
        output = model(data)
        label = data.y.to(torch.float).view(-1, 1).to(device)
        loss = F.mse_loss(output, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
        targets.append(label.cpu())
        predictions.append(output.cpu())
    return process_results(total_loss, targets, predictions, loader)

def evaluate(model, loader, device):
    model.eval()
    total_loss = 0
    targets = []
    predictions = []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            data.x = data.x.float()
            output = model(data)            
            label = data.y.to(torch.float).view(-1, 1).to(device)
            loss = F.mse_loss(output, label)
            total_loss += loss.item() * data.num_graphs
            targets.append(label.cpu())
            predictions.append(output.cpu())
    return process_results(total_loss, targets, predictions, loader)

def process_results(total_loss, targets, predictions, loader):
    targets = torch.cat(targets, dim=0)
    predictions = torch.cat(predictions, dim=0)
    rmse = torch.sqrt(F.mse_loss(predictions, targets))
    mae = torch.mean(torch.abs(predictions - targets))
    r2 = r2_score(predictions, targets)
    pcc = pcc_score(predictions, targets)
    return total_loss / len(loader.dataset), rmse, mae, r2, pcc


def save_best_results(best_metrics, best_weights, best_results_path, model_path):
    best_results = {
        'epoch': [best_metrics['epoch']],
        'best_val_loss': [best_metrics['best_val_loss']],
        'best_val_rmse': [best_metrics['best_val_rmse']],
        'best_val_mae': [best_metrics['best_val_mae']],
        'best_val_r2': [best_metrics['best_val_r2']],
        'best_val_pcc': [best_metrics['best_val_pcc']],
        'best_train_loss': [best_metrics['best_train_loss']],
        'best_train_rmse': [best_metrics['best_train_rmse']],
        'best_train_mae': [best_metrics['best_train_mae']],
        'best_train_r2': [best_metrics['best_train_r2']],
        'best_train_pcc': [best_metrics['best_train_pcc']],
        'best_test_loss': [best_metrics['best_test_loss']],
        'best_test_rmse': [best_metrics['best_test_rmse']],
        'best_test_mae': [best_metrics['best_test_mae']],
        'best_test_r2': [best_metrics['best_test_r2']],
        'best_test_pcc': [best_metrics['best_test_pcc']],
    }

    best_results_df = pd.DataFrame.from_dict(best_results)
    best_results_df.to_csv(best_results_path, index=False)

    torch.save(best_weights, model_path)

def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    

In [4]:
def main(data_path, results_path, best_results_path, model_path):
    set_seed(42)
    train_data = torch.load(os.path.join(data_path, "train.pt"))
    val_data = torch.load(os.path.join(data_path, "valid.pt"))
    test_data = torch.load(os.path.join(data_path, "test.pt"))

    train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=128, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = GraphSAGE(num_node_features=145, num_classes=1).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    early_stopping = EarlyStopping(patience=50, verbose=True)

    results = {
        'epoch': [],
        'train_loss': [],
        'train_rmse': [],
        'train_mae': [],
        'val_loss': [],
        'val_rmse': [],
        'val_mae': [],
        'test_loss': [],
        'test_rmse': [],
        'test_mae': [],
        'train_r2': [],
        'train_pcc': [],
        'val_r2': [],
        'val_pcc': [],
        'test_r2': [],
        'test_pcc': []
    }

    best_metrics = {
        'best_val_loss': float('inf'),
        'best_val_rmse': float('inf'),
        'best_val_mae': float('inf'),
        'best_val_r2': -float('inf'),
        'best_val_pcc': -float('inf'),
        'best_train_loss': None,
        'best_train_rmse': None,
        'best_train_mae': None,
        'best_train_r2': None,
        'best_train_pcc': None,
        'best_test_loss': None,
        'best_test_rmse': None,
        'best_test_mae': None,
        'best_test_r2': None,
        'best_test_pcc': None,
        'epoch': -1
    }

    for epoch in range(1000):
        train_loss, train_rmse, train_mae, train_r2, train_pcc = train(model, optimizer, train_loader, device)
        val_loss, val_rmse, val_mae, val_r2, val_pcc = evaluate(model, val_loader, device)
        test_loss, test_rmse, test_mae, test_r2, test_pcc = evaluate(model, test_loader, device)

        results['epoch'].append(epoch)
        results['train_loss'].append(round(train_loss, 3))
        results['train_rmse'].append(round(train_rmse.item(), 3))
        results['train_mae'].append(round(train_mae.item(), 3))
        results['val_loss'].append(round(val_loss, 3))
        results['val_rmse'].append(round(val_rmse.item(), 3))
        results['val_mae'].append(round(val_mae.item(), 3))
        results['test_loss'].append(round(test_loss, 3))
        results['test_rmse'].append(round(test_rmse.item(), 3))
        results['test_mae'].append(round(test_mae.item(), 3))
        results['train_r2'].append(round(train_r2.item(), 3))
        results['train_pcc'].append(round(train_pcc, 3))
        results['val_r2'].append(round(val_r2.item(), 3))
        results['val_pcc'].append(round(val_pcc, 3))
        results['test_r2'].append(round(test_r2.item(), 3))
        results['test_pcc'].append(round(test_pcc, 3))

        print(f'Epoch {epoch}: Train Loss: {train_loss:.3f}, RMSE: {train_rmse:.3f}, MAE: {train_mae:.3f}, R2: {train_r2:.3f}, PCC: {train_pcc:.3f}')
        print(f'Epoch {epoch}: Validation Loss: {val_loss:.3f}, RMSE: {val_rmse:.3f}, MAE: {val_mae:.3f}, R2: {val_r2:.3f}, PCC: {val_pcc:.3f}')
        print(f'Epoch {epoch}: Test Loss: {test_loss:.3f}, RMSE: {test_rmse:.3f}, MAE: {test_mae:.3f}, R2: {test_r2:.3f}, PCC: {test_pcc:.3f}')

        early_stopping(val_loss, model, epoch)
        if early_stopping.early_stop:
            print("Early stopping")
            break

        if val_loss < best_metrics['best_val_loss']:
            best_metrics['best_val_loss'] = val_loss
            best_metrics['best_val_rmse'] = val_rmse.item()
            best_metrics['best_val_mae'] = val_mae.item()
            best_metrics['best_val_r2'] = val_r2.item()
            best_metrics['best_val_pcc'] = val_pcc
            best_metrics['best_train_loss'] = train_loss
            best_metrics['best_train_rmse'] = train_rmse.item()
            best_metrics['best_train_mae'] = train_mae.item()
            best_metrics['best_train_r2'] = train_r2.item()
            best_metrics['best_train_pcc'] = train_pcc
            best_metrics['best_test_loss'] = test_loss
            best_metrics['best_test_rmse'] = test_rmse.item()
            best_metrics['best_test_mae'] = test_mae.item()
            best_metrics['best_test_r2'] = test_r2.item()
            best_metrics['best_test_pcc'] = test_pcc
            best_metrics['epoch'] = epoch
            best_weights = model.state_dict()

    results_df = pd.DataFrame.from_dict(results)
    results_df.to_csv(results_path, index=False)

    torch.save(model.state_dict(), 'model.pth')
    save_best_results(best_metrics, best_weights, best_results_path, model_path)

## GraphSAGE training and evaluation

In [5]:
base_dir = "/mnt/USR_DATA/ChenGeng/Project/SATCMF/Dataset/10_fold_cv/"
result_base_dir = "/mnt/USR_DATA/ChenGeng/Project/SATCMF/Dataset/"

for fold in glob.glob(base_dir + "*"):
    folder_name = fold.split("/")[-1]
    result_dir = os.path.join(result_base_dir, 'base_gnn_fold_result', folder_name)

    if not os.path.exists(result_dir):
        os.makedirs(result_dir)

    results_path = os.path.join(result_dir, folder_name + "_GraphSAGE_result.csv")
    best_results_path = os.path.join(result_dir, folder_name + "_GraphSAGE_best_result.csv")
    model_path = os.path.join(result_dir, folder_name + "_GraphSAGE_best_model.pth")

    main(fold, results_path, best_results_path, model_path)

/mnt/USR_DATA/ChenGeng/anaconda3/envs/chemprop/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 0: Train Loss: 141.107, RMSE: 11.879, MAE: 10.203, R2: -2.840, PCC: -0.092
Epoch 0: Validation Loss: 128.332, RMSE: 11.328, MAE: 9.739, R2: -2.844, PCC: -0.272
Epoch 0: Test Loss: 152.895, RMSE: 12.365, MAE: 10.546, R2: -2.680, PCC: -0.282
Validation loss decreased (inf --> 128.332134). Saving model ...
Epoch 1: Train Loss: 118.717, RMSE: 10.896, MAE: 9.021, R2: -2.231, PCC: -0.126
Epoch 1: Validation Loss: 110.035, RMSE: 10.490, MAE: 8.627, R2: -2.296, PCC: -0.340
Epoch 1: Test Loss: 133.018, RMSE: 11.533, MAE: 9.469, R2: -2.202, PCC: -0.284
Validation loss decreased (128.332134 --> 110.034752). Saving model ...
Epoch 2: Train Loss: 65.778, RMSE: 8.110, MAE: 6.348, R2: -0.790, PCC: 0.079
Epoch 2: Validation Loss: 45.482, RMSE: 6.744, MAE: 5.236, R2: -0.362, PCC: 0.374
Epoch 2: Test Loss: 57.857, RMSE: 7.606, MAE: 5.745, R2: -0.393, PCC: 0.387
Validation loss decreased (110.034752 --> 45.482000). Saving model ...
Epoch 3: Train Loss: 29.280, RMSE: 5.411, MAE: 4.102, R2: 0.203, PC

In [7]:
result_base_dir = "/mnt/USR_DATA/ChenGeng/Project/SATCMF/Dataset/"
result_dir = os.path.join(result_base_dir, 'base_gnn_fold_result')

all_data = pd.DataFrame()

for folder_name in os.listdir(result_dir):
    best_results_file = os.path.join(result_dir, folder_name, folder_name + "_GraphSAGE_best_result.csv")
    if os.path.isfile(best_results_file):
        df = pd.read_csv(best_results_file)
        all_data = pd.concat([all_data, df])

stats = all_data.agg(['mean', 'std']).transpose()

stats['formatted'] = stats.apply(lambda x: f"{x['mean']:.3f} ± {x['std']:.3f}", axis=1)
print(stats)

formatted_stats = stats[['formatted']]
formatted_stats = formatted_stats.transpose()

stats_output_path = os.path.join(result_base_dir, "GraphSAGE_aggregate_formatted_stats.csv")
formatted_stats.to_csv(stats_output_path)
print(f"Saved formatted aggregate statistics to {stats_output_path}")

                       mean        std         formatted
epoch            185.400000  52.747196  185.400 ± 52.747
best_val_loss      2.813625   0.741078     2.814 ± 0.741
best_val_rmse      1.663791   0.224662     1.664 ± 0.225
best_val_mae       1.031578   0.124348     1.032 ± 0.124
best_val_r2        0.920013   0.023150     0.920 ± 0.023
best_val_pcc       0.960601   0.011356     0.961 ± 0.011
best_train_loss    2.035032   0.370869     2.035 ± 0.371
best_train_rmse    1.421376   0.127903     1.421 ± 0.128
best_train_mae     0.998333   0.111058     0.998 ± 0.111
best_train_r2      0.945109   0.010063     0.945 ± 0.010
best_train_pcc     0.972221   0.005198     0.972 ± 0.005
best_test_loss     3.420450   0.984299     3.420 ± 0.984
best_test_rmse     1.832108   0.266311     1.832 ± 0.266
best_test_mae      1.112900   0.128008     1.113 ± 0.128
best_test_r2       0.906907   0.025942     0.907 ± 0.026
best_test_pcc      0.953654   0.013264     0.954 ± 0.013
Saved formatted aggregate stati

In [9]:
import pandas as pd

# Read the CSV file
file_path = '/mnt/USR_DATA/ChenGeng/Project/SATCMF/Dataset/GraphSAGE_aggregate_formatted_stats.csv'
df = pd.read_csv(file_path)

# Specify the columns to extract
metric_columns = [
    'best_train_rmse', 'best_train_mae', 'best_train_r2', 'best_train_pcc',
    'best_test_rmse', 'best_test_mae', 'best_test_r2', 'best_test_pcc'
]
new_columns = [
    'train_rmse', 'train_mae', 'train_r2', 'train_pcc',
    'test_rmse', 'test_mae', 'test_r2', 'test_pcc'
]

# Extract the label from the file path
label = file_path.split("/")[-1].split("_")[0]

# Create a new DataFrame to store the extracted data
new_df = pd.DataFrame()
new_df['label'] = [label] * len(df)  # Assume each row has the same label

# Copy the specified columns from the original data to the new DataFrame
for old_col, new_col in zip(metric_columns, new_columns):
    new_df[new_col] = df[old_col]

# Save to a new CSV file
new_file_path = '/mnt/USR_DATA/ChenGeng/Project/SATCMF/Dataset/GraphSAGE_aggregate_stats_cleaned.csv'
new_df.to_csv(new_file_path, index=False)
